# 🛡️ Phase 5 : Simulation Temps Réel & Dashboard

**Objectif** : Simuler le déploiement de l'IDS. Nous allons créer un flux de données "infini" à partir de notre dataset de test et voir l'IDS réagir paquet par paquet.

---

In [ ]:
import time
import pandas as pd
import numpy as np
import sys
import os
from IPython.display import display, clear_output

sys.path.append(os.path.abspath('../../'))
from src.hybrid_ids import HybridIDS

## 1. Préparation du "Faux" Temps Réel

In [ ]:
# Charger le modèle
ids = HybridIDS.load('../../models/hybrid_ids_system.pkl')

# Charger un mix de données pour la démo
df = pd.read_csv('../../data/processed/cicids2017_cleaned.csv')

# Prendre 100 paquets au hasard (mélange normal et attaques)
stream_data = df.sample(n=50, random_state=42).reset_index(drop=True)

# Préparer les features (exclure les labels)
features_to_predict = stream_data.drop(['Attack Type', 'Binary_Label', 'Attack_Merged', 'Multiclass_Label'], axis=1, errors='ignore')

## 2. Boucle de Simulation

On va itérer sur chaque ligne comme si elle arrivait du réseau.

In [ ]:
print("🟢 Démarrage du NetGuardian-AI (Simulation)...")
print("------------------------------------------------")

history = []

for i in range(len(stream_data)):
    # 1. Extraire 1 paquet (row)
    packet_features = features_to_predict.iloc[i:i+1]
    
    # 2. Prédire
    prediction = ids.predict_single(packet_features)
    
    # 3. Afficher l'alerte si nécessaire
    if prediction['is_attack']:
        msg = ids.get_alert_message(prediction)
        # Colorier en rouge pour Jupyter (ANSI codes)
        print(f"\033[91m [PACKET {i}] {msg} \033[0m")
    else:
        print(f"\033[92m [PACKET {i}] ✅ Normal \033[0m")
        
    # Historique pour stats
    history.append(prediction['type'])
    
    # 4. Pause pour effet "Temps Réel" (0.1s)
    time.sleep(0.1)
    
    # Nettoyer l'affichage tous les 10 paquets pour ne pas spammer
    if i % 20 == 0 and i > 0:
        clear_output(wait=True)
        print(f"🟢 Monitoring en cours... Paquets traités : {i}")
        
print("------------------------------------------------")
print("🏁 Simulation terminée.")

## 3. Statistiques de la Session

In [ ]:
from collections import Counter
stats = Counter(history)

print("📊 Résumé de la session :")
for det_type, count in stats.items():
    print(f" - {det_type}: {count}")